In [1]:
# Imports:
import os
import random

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import VGG19_Weights

# Project utilities:
import utils
import preprocessing
import optuna
import wandb

C:\Users\TEMP.TAU.000\.conda\envs\DeepLearning_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set seed
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
# torch.backends.cudnn.deterministic = True
# torch.use_deterministic_algorithms = True


ROOT = './data'
DATASET_DIR = './data/Post_Impressionism'
CSV_PATH = './data/classes.csv'

if not (os.path.exists(DATASET_DIR) and os.path.exists(CSV_PATH)):
    raise FileExistsError("File doesn't exist, we expect a data folder with the labels and the images folder as in the lab")


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224, 0.225])]) # Normalize the pixel values based on ImageNet statistics, to a range that VGG expects

dataset = preprocessing.ImageFolderForBinaryClassification(root=ROOT, target='is_van_gogh', transform=transform)
# Take only pictures that are in inside the csv
pics_in_csv = [i for i in range(len(dataset)) if dataset.samples[i][1] >= 0]
dataset = Subset(dataset, pics_in_csv)

In [5]:
classes = pd.read_csv(CSV_PATH)
train_classes = classes[classes['subset'] == 'train']
test_classes = classes[classes['subset'] == 'test']

train_indices, val_indices = train_test_split(train_classes.index.to_list(), test_size=0.2, random_state=SEED)
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

classes.head()

,filename,artist,genre,description,phash,width,height,genre_count,subset,is_van_gogh
0,Post_Impressionism/edouard-cortes_the-theater-...,edouard cortes,['Post Impressionism'],the-theater-of-the-comedie-francaise,9491ada9caf05cf1,1675,1382,1,train,0
1,Post_Impressionism/edouard-cortes_theatre-du-c...,edouard cortes,['Post Impressionism'],theatre-du-chatelet-1,c7d69030996f36e4,1896,1382,1,train,0
2,Post_Impressionism/edouard-vuillard_boulevard-...,edouard vuillard,['Post Impressionism'],boulevard-of-battignolles,eb7214d866c638b5,1688,1382,1,train,0
3,Post_Impressionism/edouard-vuillard_figures-ea...,edouard vuillard,['Post Impressionism'],figures-eating-in-a-garden-by-the-water,d3272568d0a95e3d,1684,1382,1,train,0
4,Post_Impressionism/edouard-vuillard_sacha-guit...,edouard vuillard,['Post Impressionism'],sacha-guitry-in-his-dressing-room-1912,dae0254af31a6fa4,1818,1382,1,train,0


### Data Augmentation

In [6]:
n_times = 25
dropout_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    *([transforms.RandomErasing(p=0.5, scale=(0.01, 0.01), ratio=(1, 1))]*n_times),
    transforms.Grayscale(num_output_channels=3),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224, 0.225])
])
dropout_dataset = Subset(preprocessing.ImageFolderForBinaryClassification(root=ROOT, transform=dropout_transform, target='is_van_gogh'), train_indices)
augmented_train_dataset = ConcatDataset([dropout_dataset, train_dataset])

In [7]:
train_loader = DataLoader(augmented_train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Fine tuning VGG19

In [8]:
# Load pre-trained VGG19 model
# vgg19 = models.vgg19(pretrained=True)
vgg19 = models.vgg19(weights=models.VGG19_Weights.DEFAULT).to(device)
vgg19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [12]:
from train import train_model_with_hyperparams

# Optuna objective function
def objective(trial):
    # Hyperparameter suggestions
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3) #Suggests a learning_rate value from a log-uniform distribution between 1e-5 and 1e-3 for hyperparameter optimization using Optuna.
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4) # Suggests a weight_decay value from a log-uniform distribution between 1e-6 and 1e-4 for regularization during Optuna hyperparameter optimization.
    patience = trial.suggest_int("patience", 3, 10) #I don't really like putting the patience as an hyper parameter - this is a thing that needs to be determined according to constraints. I put it here just to show that this is possible.
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16) # Basically choosing between 16,32,64

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # Load the train DataLoader with the chosen batch_size
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) # Load the val DataLoader with the chosen batch_size

    # Load the pre-trained model VGG16
    model = models.vgg19(weights=models.VGG19_Weights.DEFAULT).to(device) # pretrained=True == with the trained weights

    # Freeze layers
    for param in model.features.parameters():
        param.requires_grad = False
    # Unfreeze the last 6 layers in the features part of the VGG
    for param in model.features[-6:].parameters():
        param.requires_grad = True

    # Modify the classifier to fit our problem (10 classes)
    model.classifier[6] = nn.Linear(4096, 10) # Replaces the final layer of the VGG16 classifier with a new fully connected (nn.Linear) layer. The new wights are initialized randomly and hence will need to be trained
    model.classifier[6] = model.classifier[6].to(device)

    # Define optimizer and loss function
    criterion = nn.CrossEntropyLoss() # Classification.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Adam, like always, with the chosen parameters from Optuna

    # Initialize Weights & Biases - the values in the config are the properties of each trial.
    wandb.init(project="project-vgg19", #init == set the project and the "general" parameters
               config={
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "patience": patience,
        "batch_size": batch_size,
        "architecture": "VGG19",
        "dataset": "Post_Imp-10"
    },
    name=f"trial_{trial.number}") # The name that will be saved in the W&B platform

    # Train the model and get the best validation loss
    best_val_loss = train_model_with_hyperparams(model, train_loader, val_loader, optimizer, criterion, epochs=20, patience=patience, trial=trial, device = device) #send this trinal to the function above

    # Finish the Weights & Biases run
    wandb.finish()

    # Return best validation loss as the objective to minimize
    return best_val_loss

In [ ]:
# Define and optimize the Optuna study
study = optuna.create_study(direction = "minimize") # Specifies that the goal of the optimization is to minimize the objective function (e.g., validation loss because this is what returned from the objective).
study.optimize(objective, n_trials = 10) # 10 trials

[I 2025-01-13 20:48:43,824] A new study created in memory with name: no-name-22234ee5-ede5-474d-a79f-4131268a2588
C:\Users\TEMP.TAU.000\AppData\Local\Temp\ipykernel_20408\1195001642.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3) #Suggests a learning_rate value from a log-uniform distribution between 1e-5 and 1e-3 for hyperparameter optimization using Optuna.
C:\Users\TEMP.TAU.000\AppData\Local\Temp\ipykernel_20408\1195001642.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4) # Suggests a weight_decay value from a log-uniform dis

# Fine tuning AlexNet

In [ ]:
# Load the AlexNet model 
alexnet = models.alexnet(pretrained=True)


analysing results

# Style transfer function